<a href="https://colab.research.google.com/github/Kavya2099/Asian-games-athletics-web-scraping/blob/main/Asian_games_athletics_web_scraping_Final_work.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Websraping project on Asian games using Beautiful Soup to parse and extract information
Getting the list of countries who bagged medals in atletics from the year 1951-2022

I'll be scarping out year, country, category, gender and medals--> gold, silver, bronze deatils from wikipedia

https://en.wikipedia.org/wiki/List_of_Asian_Games_medalists_in_athletics

This is the final copy. You can view all explanations in my rough copy notebook!!

In [ ]:
!pip install requests --upgrade --quiet
import requests
!pip install beautifulsoup4 --upgrade --quiet
from bs4 import BeautifulSoup

In [ ]:
def get_info(url):
  wiki_url = url
  #getting the response from url
  response = requests.get(wiki_url)
  #getting the status code. If the status code is between 200-299, it means success
  if response.status_code != 200:
        raise Exception('Failed to load page {}'.format(wiki_url))
  page_contents = response.text
  doc = BeautifulSoup(page_contents, 'html.parser')
  return doc

In [ ]:
url='https://en.wikipedia.org/wiki/List_of_Asian_Games_medalists_in_athletics'
doc=get_info(url)

## Getting category

In [ ]:
category = doc.find_all('h2')
for i in range(1,4):
  category[i]= category[i].text[:-6]
category= category[1:4]
category

['Men', 'Women', 'Mixed']

We have 3 categories

##  Athletic sports type

In [ ]:
selection_class = 'mw-headline'

sports_tags = doc.find_all('span', { 'class': selection_class})

sports_male=[]

for i in range(1,26):
  sports_male.append(sports_tags[i].text)
sports_female=[]

for i in range(27,52):
  sports_female.append(sports_tags[i].text)

sports_mixed=[]


sports_mixed.append(sports_tags[53].text)
sports= sports_male+ sports_female +sports_mixed
len(sports)

51

totally we have 25 sports in male, 25 sports in female and 1 in mixed, which sums upto 51 sports type w.r.t categories.

## Data from wikipedia tables

In [ ]:
import pandas as pd
table=doc.find_all('table')

def get_year(year):
  year=year.split(' ')[0]
  return year

In [ ]:
def dataset():
  data= pd.DataFrame()
  for i in range(0,51):
    source=pd.read_html(str(table[i]))
    # convert list to dataframe
    atheletics=pd.DataFrame(source[0])
    if(i<=24):
      j=0
    elif(i<50 and i>24):
      j=1
    else:
      j=2
    atheletics['Category']=category[j]
    atheletics['Sports']= sports[i]
    atheletics.rename(columns = {'Games':'Year'}, inplace = True)
    atheletics['Year']= atheletics['Year'].apply(get_year)
    atheletics['Gold']=atheletics.Gold.str.extract(r'\((.*)\)')
    atheletics['Silver']=atheletics.Silver.str.extract(r'\((.*)\)')
    atheletics['Bronze']=atheletics.Bronze.str.extract(r'\((.*)\)')
    print("Appending "+sports[i]+ "in "+category[j])
    data=data.append(atheletics,ignore_index=True)

  return data


In [ ]:
data= dataset()

Appending 100 min Men
Appending 200 min Men
Appending 400 min Men
Appending 800 min Men
Appending 1500 min Men
Appending 5000 min Men
Appending 10,000 min Men
Appending 110 m hurdlesin Men
Appending 400 m hurdlesin Men
Appending 3000 m steeplechasein Men
Appending 4 × 100 m relayin Men
Appending 4 × 400 m relayin Men
Appending Marathonin Men
Appending 10,000 m walkin Men
Appending 20 km walkin Men
Appending 50 km walkin Men
Appending High jumpin Men
Appending Pole vaultin Men
Appending Long jumpin Men
Appending Triple jumpin Men
Appending Shot putin Men
Appending Discus throwin Men
Appending Hammer throwin Men
Appending Javelin throwin Men
Appending Decathlonin Men
Appending 100 min Women
Appending 200 min Women
Appending 400 min Women
Appending 800 min Women
Appending 1500 min Women
Appending 3000 min Women
Appending 5000 min Women
Appending 10,000 min Women
Appending 100 m hurdlesin Women
Appending 400 m hurdlesin Women
Appending 3000 m steeplechasein Women
Appending 4 × 100 m relayi

In [ ]:
data

,Year,Gold,Silver,Bronze,Category,Sports
0,1951,IND,JPN,JPN,Men,100 m
1,1954,PAK,PHI,IND,Men,100 m
2,1958,PAK,JPN,PHI,Men,100 m
3,1962,INA,MAL,PHI,Men,100 m
4,1966,MAL,SIN,JPN,Men,100 m
...,...,...,...,...,...,...
727,2006,KAZ,IND,IND,Women,Heptathlon
728,2010,UZB,JPN,IND,Women,Heptathlon
729,2014,UZB,CHN,UZB,Women,Heptathlon
730,2018,IND,CHN,JPN,Women,Heptathlon


Finally we have created our dataset, but I want to create another dataset which contains country codes with full country names.

Let's see how can we do that.

I'll take the list from https://en.wikipedia.org/wiki/List_of_IOC_country_codes

# Country code dataset

In [ ]:
country_url = 'https://en.wikipedia.org/wiki/List_of_IOC_country_codes'
doc=get_info(country_url)

In [ ]:
table=doc.find_all('table')
#changing table 0 to dataframe

country=pd.read_html(str(table[0]))
# convert list to dataframe
country=pd.DataFrame(country[0])
country= country[['Code',	'National Olympic Committee',	'Other codes used']]

#changing the value of first Code record as the record seems to have some extra values
country.Code[0]='AFG'

In [ ]:
country

,Code,National Olympic Committee,Other codes used
0,AFG,Afghanistan,NaN
1,ALB,Albania,NaN
2,ALG,Algeria,AGR (1964)AGL (1968 S) from Spanish Argelia
3,AND,Andorra,NaN
4,ANG,Angola,ANO (As referenced in IAAF World Championships...
...,...,...,...
201,VIE,Vietnam,VET (1964)VNM (1968–1976)
202,VIN,Saint Vincent and the Grenadines,NaN
203,YEM,Yemen,NaN
204,ZAM,Zambia,NRH (1964) as Northern Rhodesia


The reason for adding this dataset is beacause, we could see some countries have use different country codes for the specific time period.

This data set will help to identify the old and new country codes in that cases.

#Saving the datasets

In [36]:
# saving the dataframe
data.to_csv('Athletics_record.csv')

country.to_csv('Country_code.csv')

#Things I've learnt

- Basic concepts of web screaping using Beautiful soup
- Findall() to get the html tags and sections
- Using tables to extract wikipedia table infomration is very easy 


#Things to implement in future
- Scraping multi pages in wikipedia using wikipedia and urllib2 pacakages
- Including scraping of href links for the dataset
- Scraping someother sites apart from wikipedia


# References

- https://www.youtube.com/watch?v=RKsLLG-bzEY&t=4592s
- https://en.wikipedia.org/wiki/List_of_Asian_Games_medalists_in_athletics
- https://jovian.ai/aakashns-6l3/scraping-github-topics-repositories-rough
- https://www.w3resource.com/python-exercises/BeautifulSoup/python-beautifulsoup-exercise-9.php
- https://www.pylenin.com/blogs/python-beautiful-soup/
- https://medium.com/analytics-vidhya/web-scraping-a-wikipedia-table-into-a-dataframe-c52617e1f451
- https://www.appsloveworld.com/pandas/100/144/how-to-extract-only-the-string-component-between-parenthesis
-https://en.wikipedia.org/wiki/List_of_IOC_country_codes